In [1]:
import pandas as pd
import numpy as np


In [3]:
orders_df = pd.read_excel('global_superstore_2016.xlsx', sheet_name='Orders')
people_df = pd.read_excel('global_superstore_2016.xlsx', sheet_name='People')
returns_df = pd.read_excel('global_superstore_2016.xlsx', sheet_name='Returns')

In [4]:
print("Orders columns:", orders_df.columns)
print("People columns:", people_df.columns)
print("Returns columns:", returns_df.columns)


Orders columns: Index(['Row ID', 'Order ID', 'Order Date', 'Ship Date', 'Ship Mode',
       'Customer ID', 'Customer Name', 'Segment', 'Postal Code', 'City',
       'State', 'Country', 'Region', 'Market', 'Product ID', 'Category',
       'Sub-Category', 'Product Name', 'Sales', 'Quantity', 'Discount',
       'Profit', 'Shipping Cost', 'Order Priority'],
      dtype='object')
People columns: Index(['Person', 'Region'], dtype='object')
Returns columns: Index(['Returned', 'Order ID', 'Region'], dtype='object')


#### Data Cleaning & Transformations

In [ ]:
orders_df.dropna(subset=['Order ID', 'Sales', 'Profit'], inplace=True)


In [ ]:
orders_df['Order Date'] = pd.to_datetime(orders_df['Order Date'], errors='coerce')
orders_df['Ship Date'] = pd.to_datetime(orders_df['Ship Date'], errors='coerce')


In [ ]:
orders_df['Order_Year'] = orders_df['Order Date'].dt.year
orders_df['Order_Month'] = orders_df['Order Date'].dt.month

In [ ]:
orders_df['Segment'] = orders_df['Segment'].str.title().str.strip()

In [ ]:
orders_df['Sales'] = orders_df['Sales'].clip(lower=0)  # no negative sales
orders_df['Profit'] = orders_df['Profit'].clip(lower=-10000, upper=100000)  

In [ ]:
people_df.rename(columns={'Region': 'Region', 'Person': 'Manager'}, inplace=True)


In [ ]:
orders_merged = pd.merge(orders_df, people_df, on='Region', how='left')


In [16]:
returns_df.rename(columns={'Order ID': 'Order ID', 'Returned': 'Return_Status'}, inplace=True)


In [17]:
full_df = pd.merge(orders_merged, returns_df[['Order ID', 'Return_Status']],
                   on='Order ID', how='left')

In [18]:
full_df['Return_Status'] = full_df['Return_Status'].fillna('Not Returned')


In [20]:
print("Final shape:", full_df.shape)
full_df.head()

Final shape: (51290, 28)


,Row ID,Order ID,Order Date,Ship Date,Ship Mode,Customer ID,Customer Name,Segment,Postal Code,City,...,Sales,Quantity,Discount,Profit,Shipping Cost,Order Priority,Order_Year,Order_Month,Manager,Return_Status
0,40098,CA-2014-AB10015140-41954,2014-11-11,2014-11-13,First Class,AB-100151402,Aaron Bergman,Consumer,73120.0,Oklahoma City,...,221.980,2,0.0,62.1544,40.77,High,2014,11,Lon Bonher,Not Returned
1,26341,IN-2014-JR162107-41675,2014-02-05,2014-02-07,Second Class,JR-162107,Justin Ritter,Corporate,NaN,Wollongong,...,3709.395,9,0.1,-288.7650,923.63,Critical,2014,2,Kauri Anaru,Not Returned
2,25330,IN-2014-CR127307-41929,2014-10-17,2014-10-18,First Class,CR-127307,Craig Reiter,Consumer,NaN,Brisbane,...,5175.171,9,0.1,919.9710,915.49,Medium,2014,10,Kauri Anaru,Not Returned
3,13524,ES-2014-KM1637548-41667,2014-01-28,2014-01-30,First Class,KM-1637548,Katherine Murray,Home Office,NaN,Berlin,...,2892.510,5,0.1,-96.5400,910.16,Medium,2014,1,Gilbert Wolff,Not Returned
4,47221,SG-2014-RH9495111-41948,2014-11-05,2014-11-06,Same Day,RH-9495111,Rick Hansen,Consumer,NaN,Dakar,...,2832.960,8,0.0,311.5200,903.04,Critical,2014,11,Katlego Akosua,Not Returned


In [21]:
full_df.to_csv('GlobalSuperstore_cleaned.csv', index=False)
